In [1]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from safetensors.torch import load_file
import torch
from sagemaker.serve.mode.function_pointers import Mode
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.serve import InferenceSpec
import boto3
from sagemaker.serve import CustomPayloadTranslator
import torch
from torchvision.transforms import transforms
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline
from sagemaker.serve import ModelServer
import numpy as np
import io
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-02-20 07:06:32.723843: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Using Sagemaker's 'ModelBuilder' Class to Format and Deploy the PyTorch Model

In [2]:
sample_input = 'Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand. The story should use the verb "buy", the noun "passport" and the adjective "pretty". The story has the following features: the story should contain at least one dialogue. Remember to only use simple words!'

In [6]:
sample_output = "Once upon a time, there was a pretty girl named Lily. She loved to travel and see new places. One day, Lily decided she wanted to go on a big trip. But first, she needed to buy a passport. Lily went to the passport place with her mom. She said, 'Mom, can you help me buy a passport, please?' Her mom smiled and said, 'Of course, Lily. Let's do it together.' They filled out the papers and took a nice photo. A few days later, Lily got her passport in the mail. She was so happy! Now she could go on her big trip. She packed her bags and said goodbye to her friends. With her pretty new passport, Lily was ready to explore the world."

## Inference Object for Handling Input/Output of Model

In [4]:
# custom inference spec with hugging face pipeline
class InferenceSpec(InferenceSpec):
    def load(self, model_dir: str):
        return pipeline(model="Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k")
        
    def invoke(self, input, model):
        return model(input)
    
    
inf_spec = InferenceSpec()

In [5]:
instance_type = 'ml.g4dn.4xlarge'
image = image_uris.retrieve(region=region, framework='huggingface', image_scope='inference', version='4.28.1', base_framework_version='pytorch2.0.0', instance_type=instance_type)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py310


In [6]:
value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

In [7]:
from sagemaker.huggingface.model import HuggingFaceModel

In [64]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    inference_spec=inf_spec,
    role_arn=execution_role,
    image_uri=image,
    schema_builder=schema,
    env_vars={"HF_TASK":"text-generation"},
    model_server=ModelServer.TORCHSERVE
)

In [65]:
model_builder

ModelBuilder(model_path='/tmp/sagemaker/model-builder/ceb062beef4211efa3eaaa57039344d3', role_arn='arn:aws:iam::597161074694:role/service-role/SageMaker-ExecutionRole-20250210T145384', sagemaker_session=None, name='model-name-ceb063c2ef4211efa3eaaa57039344d3', mode=<Mode.SAGEMAKER_ENDPOINT: 3>, shared_libs=[], dependencies={'auto': False}, env_vars={'HF_TASK': 'text-generation'}, log_level=10, content_type=None, accept_type=None, s3_model_data_url=None, instance_type='ml.c5.xlarge', schema_builder=SchemaBuilder(
input_serializer=<sagemaker.base_serializers.StringSerializer object at 0x7f2df8cf5dd0>
output_serializer=<sagemaker.serve.builder.schema_builder.JSONSerializerWrapper object at 0x7f2df5dd4050>
input_deserializer=<sagemaker.base_deserializers.StringDeserializer object at 0x7f2e03f57cd0>
output_deserializer=<sagemaker.base_deserializers.JSONDeserializer object at 0x7f2f94906510>), model=None, inference_spec=<__main__.InferenceSpec object at 0x7f2e1b0b27d0>, image_uri='7631043518

In [26]:
aws_model = model_builder.build(role_arn=execution_role, sagemaker_session=sagemaker_session)

ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: INFO:     Skipping auto detection as the image uri is provided 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04
ModelBuilder: DEBUG:     Uploading the model resources to bucket=sagemaker-us-east-1-597161074694, key_prefix=huggingface-pytorch-inference-2025-02-20-05-01-33-690.
Uploading model artifacts: 100%|██████████████████████████| 8546/8546 [00:00<00:00, 67562.08bytes/s]
ModelBuilder: DEBUG:     Model resources uploaded to: s3://sagemaker-us-east-1-597161074694/huggingface-pytorch-inference-2025-02-20-05-01-33-690/serve.tar.gz


In [ ]:
predictor = aws_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.4xlarge",
)

In [29]:
predictor.predict('Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand. The story should use the verb "buy", the noun "passport" and the adjective "pretty". The story has the following features: the story should contain at least one dialogue. Remember to only use simple words!')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "model_fn() takes 1 positional argument but 2 were given"
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2025-02-20-05-01-39-355 in account 597161074694 for more information.